In [1]:
# 1）测试 layers.input_from_feature_columns  
# ref link: https://zhuanlan.zhihu.com/p/393327229

import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib.layers.python.layers.feature_column import _EmbeddingColumn 
from tensorflow.contrib.layers.python.layers import initializers

from collections import OrderedDict
import numpy as np
import json

# dense feature ====================================================================== #
# ops
discount_rate_column = layers.real_valued_column(
		column_name=str('discount_rate'),
		dimension=1,
		default_value=0.0
	)

cvr_column = layers.real_valued_column(
		column_name=str('cvr'),
		dimension=1,
		default_value=0.0
		)	

rating_column = layers.real_valued_column(
		column_name=str('rating'),
		dimension=1,
		default_value=0.0
		)
# concat ops
real_value_feature_column = [discount_rate_column, rating_column]

# sparse feature ====================================================================== #
# ops
embedding_hash_column = layers.sparse_column_with_hash_bucket(
                column_name=str('refer_type'),
                hash_bucket_size=1000
                )
embedding_column = _EmbeddingColumn(embedding_hash_column, 
				dimension=10, 
				combiner='mean', 
				max_norm=None,
				initializer=initializers.xavier_initializer()
                )
# concat ops
embedding_feature_column = [embedding_column]

# ======== hyper ops ============================================================== #
hyper_feature_column = [discount_rate_column, rating_column, embedding_column]

# ======== 真实输入数据 + ops (input_from_feature_columns) ========================== #
# for dense 
t1 = tf.Variable(tf.zeros([4,1]), dtype=tf.float32)
t2 = tf.Variable(tf.ones([4,1]), dtype=tf.float32)
t3 = tf.Variable(2*tf.ones([4,1]), dtype=tf.float32)
fg_real_value_dict = {'discount_rate':t1, 'cvr':t2, 'rating':t3}
#
input_layer_real_value = layers.input_from_feature_columns(fg_real_value_dict, real_value_feature_column)

# for sparse
refer_type_column = tf.SparseTensor(indices=tf.constant([[0,0],[1,0],[2,0],[3,0]],dtype=tf.int64), 
		values=tf.constant(['R','B','R','G'],dtype=tf.string), dense_shape=tf.constant([4,2], dtype=tf.int64))
fg_embedding_dict = {'refer_type':refer_type_column}
#
input_layer_sparse_embedding = layers.input_from_feature_columns(fg_embedding_dict, embedding_feature_column)

# for hyper
hyper_fg_features_dict = {'discount_rate':t1, 'cvr':t2, 'rating':t3, 'refer_type':refer_type_column}
#
input_layer_features_emb = layers.input_from_feature_columns(hyper_fg_features_dict, hyper_feature_column)

# ======= run ops + 测试效果 ======= #
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    #
    prt_v1 = sess.run(input_layer_real_value)
    prt_v2 = sess.run(input_layer_sparse_embedding)
    prt_v3 = sess.run(input_layer_features_emb)
    #
    print("v1 shape:", input_layer_real_value.get_shape().as_list())
    print("v2 shape:", input_layer_sparse_embedding.get_shape().as_list())
    print("v3 shape:", input_layer_features_emb.get_shape().as_list()) # 12 = (1+1) + (10), 2个dense + 1个sparse_emb
    print("v1:", prt_v1)
    print("v2:", prt_v2)
    print("v3:", prt_v3)

v1 shape: [4, 2]
v2 shape: [4, 10]
v3 shape: [4, 12]
v1: [[0. 2.]
 [0. 2.]
 [0. 2.]
 [0. 2.]]
v2: [[-0.02515069 -0.024638    0.04365104 -0.07213864 -0.00868414 -0.06193081
   0.05852105 -0.06890333 -0.01705331 -0.0252091 ]
 [-0.05071261  0.03064359  0.06124937  0.01807063 -0.05551904  0.03919241
   0.01864873  0.05486334 -0.00930299 -0.02780403]
 [-0.02515069 -0.024638    0.04365104 -0.07213864 -0.00868414 -0.06193081
   0.05852105 -0.06890333 -0.01705331 -0.0252091 ]
 [ 0.04916567 -0.00744983  0.04231632 -0.00395001 -0.03115816  0.01171283
   0.03413805 -0.03945787  0.07215839 -0.02736607]]
v3: [[-3.4786239e-02 -6.9296375e-02 -2.0847581e-02  6.5376170e-02
   5.3190283e-02 -6.7934170e-02  6.0509138e-02  6.3413925e-02
  -2.4580210e-04 -6.8562694e-02  0.0000000e+00  2.0000000e+00]
 [-3.4718875e-02 -4.7171652e-02  4.9681775e-02 -4.2835940e-02
  -1.2355678e-02 -3.6193229e-02  4.0258870e-02  4.9290210e-03
   5.9730105e-02  2.5950298e-02  0.0000000e+00  2.0000000e+00]
 [-3.4786239e-02 -6.929